
# ✈️ Build Flight Datasets (Airports + Routes) — **Offline Version**

This notebook **does not use the internet**. Place the required source files in a folder (default: `data_in/`), then run the cells.

## 📦 Required local files (put inside `data_in/`)
- `airports.csv`   — OurAirports airports master
- `countries.csv`  — OurAirports countries
- `regions.csv`    — OurAirports regions
- `airports.dat`   — OpenFlights airports table
- `routes.dat`     — OpenFlights routes table

> Tip: You can rename the input folder by changing `DATA_IN` below.


## 1) Setup paths

In [1]:

import os, numpy as np, pandas as pd

DATA_IN  = os.path.join(os.getcwd(), "data_in")   # change if needed
DATA_OUT = os.path.join(os.getcwd(), "data_out")
os.makedirs(DATA_OUT, exist_ok=True)

required_files = ["airports.csv", "countries.csv", "regions.csv", "airports.dat", "routes.dat"]
missing = [f for f in required_files if not os.path.exists(os.path.join(DATA_IN, f))]
if missing:
    raise FileNotFoundError(f"Missing files in {DATA_IN}: {missing}\nPlease add them and re-run.")
print("✅ All required files found in:", DATA_IN)


✅ All required files found in: c:\Projects\flight analysis\data_in


## 2) Load sources (offline)

In [3]:

ap_oa = pd.read_csv(os.path.join(DATA_IN, "airports.csv"), low_memory=False)
countries = pd.read_csv(os.path.join(DATA_IN, "countries.csv"), low_memory=False)
regions   = pd.read_csv(os.path.join(DATA_IN, "regions.csv"), low_memory=False)

ap_of = pd.read_csv(os.path.join(DATA_IN, "airports.dat"), header=None, dtype=str)
ap_of.columns = ["airport_id","name","city","country","iata","icao",
                 "lat","lon","alt_ft","timezone","dst","tzdb","type","source"]
ap_of["airport_id"] = pd.to_numeric(ap_of["airport_id"], errors="coerce")
for c in ["iata","icao","name","country","city"]:
    ap_of[c] = ap_of[c].astype(str).str.strip().str.upper().replace({"\\N": np.nan, "": np.nan})
for c in ["lat","lon"]:
    ap_of[c] = pd.to_numeric(ap_of[c], errors="coerce")

routes = pd.read_csv(os.path.join(DATA_IN, "routes.dat"), header=None, dtype=str)
routes.columns = ["airline","airline_id","src_code","src_id",
                  "dst_code","dst_id","codeshare","stops","equipment"]
for col in ["airline","src_code","dst_code"]:
    routes[col] = routes[col].astype(str).str.strip().str.upper()
routes["src_id"] = pd.to_numeric(routes["src_id"], errors="coerce")
routes["dst_id"] = pd.to_numeric(routes["dst_id"], errors="coerce")

ap_oa.head(), ap_of.head(), routes.head()


(       id ident           type                  name  latitude_deg  \
 0    6523   00A       heliport     Total RF Heliport     40.070985   
 1  323361  00AA  small_airport  Aero B Ranch Airport     38.704022   
 2    6524  00AK  small_airport          Lowell Field     59.947733   
 3    6525  00AL  small_airport          Epps Airpark     34.864799   
 4  506791  00AN  small_airport  Katmai Lodge Airport     59.093287   
 
    longitude_deg  elevation_ft continent iso_country iso_region  municipality  \
 0     -74.933689          11.0       NaN          US      US-PA      Bensalem   
 1    -101.473911        3435.0       NaN          US      US-KS         Leoti   
 2    -151.692524         450.0       NaN          US      US-AK  Anchor Point   
 3     -86.770302         820.0       NaN          US      US-AL       Harvest   
 4    -156.456699          80.0       NaN          US      US-AK   King Salmon   
 
   scheduled_service icao_code iata_code gps_code local_code  \
 0            

## 3) Normalize OurAirports schema

In [4]:

def normalize_ourairports(df: pd.DataFrame) -> pd.DataFrame:
    def clean(s):
        return s.astype(str).str.strip().str.upper().replace({'NAN': pd.NA, 'NONE': pd.NA, '': pd.NA})
    for c in ['iata_code','ident','local_code','iso_country','name','municipality']:
        if c in df.columns:
            df[c] = clean(df[c])
    df['lat'] = pd.to_numeric(df['latitude_deg'], errors='coerce')
    df['lon'] = pd.to_numeric(df['longitude_deg'], errors='coerce')
    return df

ap_oa = normalize_ourairports(ap_oa)
ap_oa[['iata_code','ident','local_code','lat','lon']].head()


,iata_code,ident,local_code,lat,lon
0,<NA>,00A,00A,40.070985,-74.933689
1,<NA>,00AA,00AA,38.704022,-101.473911
2,<NA>,00AK,00AK,59.947733,-151.692524
3,<NA>,00AL,00AL,34.864799,-86.770302
4,<NA>,00AN,00AN,59.093287,-156.456699


## 4) Merge routes with airport coordinates (multi-fallback)

In [5]:

def merge_routes_with_coords(routes: pd.DataFrame, ap_oa: pd.DataFrame, ap_of: pd.DataFrame) -> pd.DataFrame:
    r = routes.copy()

    # Pass 1: IATA (OurAirports)
    r = r.merge(
        ap_oa[['iata_code','lat','lon']].rename(columns={'iata_code':'src_code','lat':'lat_src_iata','lon':'lon_src_iata'}),
        on='src_code', how='left'
    ).merge(
        ap_oa[['iata_code','lat','lon']].rename(columns={'iata_code':'dst_code','lat':'lat_dst_iata','lon':'lon_dst_iata'}),
        on='dst_code', how='left'
    )

    # Pass 2: ICAO/ident (OurAirports)
    r = r.merge(
        ap_oa[['ident','lat','lon']].rename(columns={'ident':'src_code','lat':'lat_src_icao','lon':'lon_src_icao'}),
        on='src_code', how='left'
    ).merge(
        ap_oa[['ident','lat','lon']].rename(columns={'ident':'dst_code','lat':'lat_dst_icao','lon':'lon_dst_icao'}),
        on='dst_code', how='left'
    )

    # Coalesce
    r['lat_src'] = r['lat_src_iata'].fillna(r['lat_src_icao'])
    r['lon_src'] = r['lon_src_iata'].fillna(r['lon_src_icao'])
    r['lat_dst'] = r['lat_dst_iata'].fillna(r['lat_dst_icao'])
    r['lon_dst'] = r['lon_dst_iata'].fillna(r['lon_dst_icao'])

    # Pass 3: OpenFlights by airport_id
    r = r.merge(
        ap_of[['airport_id','lat','lon']].rename(columns={'airport_id':'src_id','lat':'lat_src_id','lon':'lon_src_id'}),
        on='src_id', how='left'
    ).merge(
        ap_of[['airport_id','lat','lon']].rename(columns={'airport_id':'dst_id','lat':'lat_dst_id','lon':'lon_dst_id'}),
        on='dst_id', how='left'
    )

    src_missing = r['lat_src'].isna() | r['lon_src'].isna()
    dst_missing = r['lat_dst'].isna() | r['lon_dst'].isna()
    r.loc[src_missing, 'lat_src'] = r.loc[src_missing, 'lat_src'].fillna(r.loc[src_missing, 'lat_src_id'])
    r.loc[src_missing, 'lon_src'] = r.loc[src_missing, 'lon_src'].fillna(r.loc[src_missing, 'lon_src_id'])
    r.loc[dst_missing, 'lat_dst'] = r.loc[dst_missing, 'lat_dst'].fillna(r.loc[dst_missing, 'lat_dst_id'])
    r.loc[dst_missing, 'lon_dst'] = r.loc[dst_missing, 'lon_dst'].fillna(r.loc[dst_missing, 'lon_dst_id'])

    # Pass 4: FAA local_code (US) & K-stripped ICAO → local
    def strip_k(code):
        if isinstance(code, str) and len(code) == 4 and code.startswith('K'):
            return code[1:]
        return code
    r['src_code_k'] = r['src_code'].apply(strip_k)
    r['dst_code_k'] = r['dst_code'].apply(strip_k)

    r = r.merge(
        ap_oa[['local_code','lat','lon']].rename(columns={'local_code':'src_code_k','lat':'lat_src_loc','lon':'lon_src_loc'}),
        on='src_code_k', how='left'
    ).merge(
        ap_oa[['local_code','lat','lon']].rename(columns={'local_code':'dst_code_k','lat':'lat_dst_loc','lon':'lon_dst_loc'}),
        on='dst_code_k', how='left'
    )

    src_missing = r['lat_src'].isna() | r['lon_src'].isna()
    dst_missing = r['lat_dst'].isna() | r['lon_dst'].isna()
    r.loc[src_missing, 'lat_src'] = r.loc[src_missing, 'lat_src'].fillna(r.loc[src_missing, 'lat_src_loc'])
    r.loc[src_missing, 'lon_src'] = r.loc[src_missing, 'lon_src'].fillna(r.loc[src_missing, 'lon_src_loc'])
    r.loc[dst_missing, 'lat_dst'] = r.loc[dst_missing, 'lat_dst'].fillna(r.loc[dst_missing, 'lat_dst_loc'])
    r.loc[dst_missing, 'lon_dst'] = r.loc[dst_missing, 'lon_dst'].fillna(r.loc[dst_missing, 'lon_dst_loc'])

    # Clean helper columns
    r = r.drop(columns=[
        'lat_src_iata','lon_src_iata','lat_dst_iata','lon_dst_iata',
        'lat_src_icao','lon_src_icao','lat_dst_icao','lon_dst_icao',
        'lat_src_id','lon_src_id','lat_dst_id','lon_dst_id',
        'lat_src_loc','lon_src_loc','lat_dst_loc','lon_dst_loc',
        'src_code_k','dst_code_k'
    ])

    return r

r = merge_routes_with_coords(routes, ap_oa, ap_of)

missing_src = r['lat_src'].isna().sum()
missing_dst = r['lat_dst'].isna().sum()
print(f"Unmatched after merges  →  source: {missing_src} | destination: {missing_dst}")


Unmatched after merges  →  source: 12 | destination: 13


## 5) Save outputs (world + India)

In [6]:

# Airports world
airports_world = ap_oa.copy()
airports_world[['name','municipality','iso_country','iata_code','ident','local_code','latitude_deg','longitude_deg']]    .to_csv(os.path.join(DATA_OUT, "airports_world.csv"), index=False)

# Airports India with state/UT
ap_india = ap_oa[ap_oa['iso_country'] == 'IN'].copy()
regions_slim = regions[['code','name']].rename(columns={'code':'iso_region','name':'state_ut'})
ap_india = ap_india.merge(regions_slim, on='iso_region', how='left')
ap_india[['name','municipality','state_ut','iso_country','iata_code','ident','local_code','latitude_deg','longitude_deg']]    .to_csv(os.path.join(DATA_OUT, "airports_india.csv"), index=False)

# Routes full + clean
r.to_csv(os.path.join(DATA_OUT, "routes_full_with_coords.csv"), index=False)
r_clean = r.dropna(subset=['lat_src','lon_src','lat_dst','lon_dst']).copy()
r_clean.to_csv(os.path.join(DATA_OUT, "routes_full_with_coords_clean.csv"), index=False)

# Add country tags & India subset
code_country = pd.concat([
    ap_oa[['iata_code','iso_country']].rename(columns={'iata_code':'code'}),
    ap_oa[['ident','iso_country']].rename(columns={'ident':'code'})
], ignore_index=True).dropna().drop_duplicates('code')

r_cc = r_clean.merge(code_country.rename(columns={'code':'src_code','iso_country':'src_country'}),
                     on='src_code', how='left')               .merge(code_country.rename(columns={'code':'dst_code','iso_country':'dst_country'}),
                     on='dst_code', how='left')

routes_india = r_cc[(r_cc['src_country'] == 'IN') | (r_cc['dst_country'] == 'IN')].copy()
routes_india.to_csv(os.path.join(DATA_OUT, "routes_india_with_coords.csv"), index=False)

print("✅ Saved to", DATA_OUT)
print("\n".join([
    "airports_world.csv",
    "airports_india.csv",
    "routes_full_with_coords.csv",
    "routes_full_with_coords_clean.csv",
    "routes_india_with_coords.csv",
]))


✅ Saved to c:\Projects\flight analysis\data_out
airports_world.csv
airports_india.csv
routes_full_with_coords.csv
routes_full_with_coords_clean.csv
routes_india_with_coords.csv


In [9]:
import plotly.io as pio
pio.renderers.default = "notebook"


In [11]:
# ===============================
# 🗺️  Quick sanity-check map
# ===============================

import plotly.graph_objects as go
import pandas as pd
import numpy as np
import os

# Load merged routes
routes_path = os.path.join(DATA_OUT, "routes_full_with_coords_clean.csv")
rmap = pd.read_csv(routes_path)

# Optional: sample to avoid plotting 60k+ arcs
rmap_sample = rmap.sample(n=min(200, len(rmap)), random_state=42)

fig = go.Figure()

# Add route arcs
for _, row in rmap_sample.iterrows():
    fig.add_trace(
        go.Scattergeo(
            lon=[row['lon_src'], row['lon_dst']],
            lat=[row['lat_src'], row['lat_dst']],
            mode='lines',
            line=dict(width=0.6, color='blue'),
            opacity=0.3
        )
    )

# Add source/destination airport markers
fig.add_trace(
    go.Scattergeo(
        lon=pd.concat([rmap_sample['lon_src'], rmap_sample['lon_dst']]),
        lat=pd.concat([rmap_sample['lat_src'], rmap_sample['lat_dst']]),
        mode='markers',
        marker=dict(size=2, color='red'),
        opacity=0.6
    )
)

fig.update_layout(
    title_text='🌍 Global Flight Routes (sample)',
    showlegend=False,
    geo=dict(
        projection_type='natural earth',
        showland=True, landcolor='lightgray',
        showcountries=True,
        countrycolor='gray',
        lataxis_showgrid=True,
        lonaxis_showgrid=True,
    )
)

fig.show(renderer="browser")


In [12]:
# ===============================
# 🇮🇳  India-only Flight Routes Map
# ===============================

import plotly.graph_objects as go
import pandas as pd
import numpy as np
import os
import plotly.io as pio

# ✅ Use browser renderer (avoids nbformat issue)
pio.renderers.default = "browser"

# Load India-only routes file created earlier
routes_india_path = os.path.join(DATA_OUT, "routes_india_with_coords.csv")
routes_in = pd.read_csv(routes_india_path)

print("Total India routes:", len(routes_in))

# Optional: sample for clarity
rmap_in = routes_in.sample(n=min(300, len(routes_in)), random_state=42)

fig_in = go.Figure()

# Add route arcs
for _, row in rmap_in.iterrows():
    fig_in.add_trace(
        go.Scattergeo(
            lon=[row['lon_src'], row['lon_dst']],
            lat=[row['lat_src'], row['lat_dst']],
            mode='lines',
            line=dict(width=0.6, color='royalblue'),
            opacity=0.4
        )
    )

# Add airports as points
fig_in.add_trace(
    go.Scattergeo(
        lon=pd.concat([rmap_in['lon_src'], rmap_in['lon_dst']]),
        lat=pd.concat([rmap_in['lat_src'], rmap_in['lat_dst']]),
        mode='markers',
        marker=dict(size=3, color='red', opacity=0.7),
        hoverinfo='skip'
    )
)

# Configure layout
fig_in.update_layout(
    title_text="🇮🇳 Indian Flight Route Network (Sample)",
    showlegend=False,
    geo=dict(
        projection_type='mercator',
        showland=True,
        landcolor='rgb(240, 240, 240)',
        showcountries=True,
        countrycolor='gray',
        lonaxis=dict(range=[65, 100]),   # approximate India region
        lataxis=dict(range=[5, 38]),
        center=dict(lat=21, lon=78),
    )
)

fig_in.show(renderer="browser")


Total India routes: 2596


In [13]:
# ===============================
# 🇮🇳  India-only Flight Routes Map (with Airport Names)
# ===============================

import plotly.graph_objects as go
import pandas as pd
import os
import plotly.io as pio

# Use browser renderer to avoid nbformat issue
pio.renderers.default = "browser"

# Load India routes
routes_india_path = os.path.join(DATA_OUT, "routes_india_with_coords.csv")
routes_in = pd.read_csv(routes_india_path)

print("Total India routes:", len(routes_in))

# Sample (optional)
rmap_in = routes_in.sample(n=min(300, len(routes_in)), random_state=42)

# Collect all airport points (source + destination)
airports_in = pd.concat([
    rmap_in[['src_code', 'lat_src', 'lon_src']].rename(columns={'src_code': 'code', 'lat_src': 'lat', 'lon_src': 'lon'}),
    rmap_in[['dst_code', 'lat_dst', 'lon_dst']].rename(columns={'dst_code': 'code', 'lat_dst': 'lat', 'lon_dst': 'lon'})
]).drop_duplicates(subset=['code'])

# Try joining airport names from your airports_india.csv (if available)
airports_path = os.path.join(DATA_OUT, "airports_india.csv")
if os.path.exists(airports_path):
    ap_india = pd.read_csv(airports_path)
    ap_india['iata_code'] = ap_india['iata_code'].astype(str).str.upper()
    airports_in = airports_in.merge(ap_india[['iata_code', 'name', 'municipality']], left_on='code', right_on='iata_code', how='left')

# Fallback: use code if name missing
airports_in['label'] = airports_in.apply(
    lambda x: f"{x['name']} ({x['code']})" if pd.notna(x.get('name')) else x['code'], axis=1
)

# Build figure
fig_in = go.Figure()

# Add flight route arcs
for _, row in rmap_in.iterrows():
    fig_in.add_trace(
        go.Scattergeo(
            lon=[row['lon_src'], row['lon_dst']],
            lat=[row['lat_src'], row['lat_dst']],
            mode='lines',
            line=dict(width=0.5, color='royalblue'),
            opacity=0.3,
            hoverinfo='skip'
        )
    )

# Add airport markers with hover labels
fig_in.add_trace(
    go.Scattergeo(
        lon=airports_in['lon'],
        lat=airports_in['lat'],
        text=airports_in['label'],
        mode='markers',
        marker=dict(size=4, color='red', opacity=0.7),
        hovertemplate="%{text}<extra></extra>",
        name='Airports'
    )
)

# Label major airports permanently
major_codes = ['DEL', 'BOM', 'BLR', 'HYD', 'MAA', 'CCU']
for _, row in airports_in[airports_in['code'].isin(major_codes)].iterrows():
    fig_in.add_trace(go.Scattergeo(
        lon=[row['lon']], lat=[row['lat']],
        text=row['code'],
        mode='text',
        textfont=dict(size=10, color='black'),
        showlegend=False
    ))

# Configure layout
fig_in.update_layout(
    title_text="🇮🇳 Indian Flight Route Network (with Airport Names)",
    showlegend=False,
    geo=dict(
        projection_type='mercator',
        showland=True, landcolor='rgb(245, 245, 245)',
        showcountries=True,
        countrycolor='gray',
        lonaxis=dict(range=[65, 100]),
        lataxis=dict(range=[5, 38]),
        center=dict(lat=21, lon=78),
    )
)

fig_in.show(renderer="browser")


Total India routes: 2596


In [14]:
# ===============================
# 🌍 World Airports + Route Arcs (layered)
# ===============================
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import os

pio.renderers.default = "browser"

DATA_IN   = os.path.join(os.getcwd(), "data_in")
DATA_OUT  = os.path.join(os.getcwd(), "data_out")

# Airports (OurAirports)
ap_oa = pd.read_csv(os.path.join(DATA_IN, "airports.csv"), low_memory=False)
valid_types = {"large_airport", "medium_airport", "small_airport"}
ap = ap_oa[ap_oa["type"].isin(valid_types)].copy()

# Optional sampling for airports
MAX_AIRPTS = 12000
if len(ap) > MAX_AIRPTS:
    ap_plot = ap.sample(MAX_AIRPTS, random_state=42)
else:
    ap_plot = ap

# Routes (your merged file)
routes_path = os.path.join(DATA_OUT, "routes_full_with_coords_clean.csv")
r = pd.read_csv(routes_path)

# Optional sampling for arcs
MAX_ARCS = 500
if len(r) > MAX_ARCS:
    r_plot = r.sample(MAX_ARCS, random_state=42)
else:
    r_plot = r

fig = go.Figure()

# Airports layer
fig.add_trace(go.Scattergeo(
    lon=ap_plot["longitude_deg"],
    lat=ap_plot["latitude_deg"],
    text=ap_plot["name"] + " (" + ap_plot["iata_code"].fillna(ap_plot["ident"]) + ")",
    mode="markers",
    marker=dict(size=2, opacity=0.7),
    hovertemplate="%{text}<extra></extra>",
    name="Airports"
))

# Route arcs layer
for _, row in r_plot.iterrows():
    fig.add_trace(go.Scattergeo(
        lon=[row["lon_src"], row["lon_dst"]],
        lat=[row["lat_src"], row["lat_dst"]],
        mode="lines",
        line=dict(width=0.5),
        opacity=0.25,
        showlegend=False
    ))

fig.update_layout(
    title_text=f"🌍 Airports + Sampled Routes — Airports: {len(ap_plot):,}, Routes: {len(r_plot):,}",
    showlegend=False,
    geo=dict(
        projection_type='natural earth',
        showland=True, landcolor='lightgray',
        showcountries=True, countrycolor='gray'
    )
)
fig.show(renderer="browser")


In [15]:
# ===============================
# 🌍 World Airports Map (OurAirports, ALL)
# ===============================
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import os

pio.renderers.default = "browser"

DATA_IN  = os.path.join(os.getcwd(), "data_in")
ap_oa = pd.read_csv(os.path.join(DATA_IN, "airports.csv"), low_memory=False)

# Keep only real airports (exclude heliports, seaplane bases, etc.)
valid_types = {"large_airport", "medium_airport", "small_airport"}  # adjust if you want more/less
ap = ap_oa[ap_oa["type"].isin(valid_types)].copy()

# Optional: show only airports with scheduled service to reduce clutter
# ap = ap[ap["scheduled_service"].str.lower().eq("yes")]

# OPTIONAL sampling for speed (Plotly can get slow with 30–40k points)
MAX_POINTS = 15000
if len(ap) > MAX_POINTS:
    ap = ap.sample(MAX_POINTS, random_state=42)

fig = go.Figure(go.Scattergeo(
    lon=ap["longitude_deg"],
    lat=ap["latitude_deg"],
    text=ap["name"] + " (" + ap["iata_code"].fillna(ap["ident"]) + ")",
    mode="markers",
    marker=dict(size=2, opacity=0.7),
    hovertemplate="%{text}<extra></extra>"
))

fig.update_layout(
    title_text=f"🌍 World Airports (OurAirports) — showing {len(ap):,} points",
    showlegend=False,
    geo=dict(
        projection_type='natural earth',
        showland=True, landcolor='lightgray',
        showcountries=True, countrycolor='gray'
    )
)
fig.show()
